In [ ]:
import math
import numpy as np
import pandas as pd
from itertools import product

"""
Diffraction Pattern Indexing App for Si (cubic) and GaN (hexagonal)

User inputs: measured distance ratio (d1/d2) and angle between two diffraction spots.
The app matches against physically allowed plane sets for each structure,
computes d-spacings, angles, and proper zone axes.
"""

def overline(n):
    """Return overline notation for negative Miller indices"""
    return f"\u0305{{{abs(n)}}}" if n < 0 else str(n)

def format_hkl(h, k, l):
    """Format cubic Miller indices (hkl) with overline"""
    return f"({overline(h)}{overline(k)}{overline(l)})"

def format_hkil(h, k, i, l):
    """Format hexagonal Miller-Bravais indices (hkil) with overline"""
    return f"({overline(h)} {overline(k)} {overline(i)} {overline(l)})"

# ---- Allowed planes ----
# Si cubic: {111, 200, 220, 311, 222, 400} and negatives
si_basic = [(1,1,1),(2,0,0),(2,2,0),(3,1,1),(2,2,2),(4,0,0)]
si_planes = []
for h,k,l in si_basic:
    for sh,sk,sl in product([1,-1], repeat=3):
        si_planes.append((h*sh, k*sk, l*sl))
si_planes = list(set(si_planes))

# GaN hexagonal: list of 4-index and sign variations
gan_basic = [(1,0,-1,0),(0,0,0,2),(1,0,-1,1),(1,0,-1,2),(1,1,-2,0),(0,0,0,1),(1,0,-1,3),(1,1,-2,2),(2,0,-2,1)]
gan_planes = []
for h,k,i,l in gan_basic:
    for sign in [1,-1]:
        gan_planes.append((h*sign, k*sign, i*sign, l*sign))
gan_planes = list(set(gan_planes))

# ---- Lattice constants ----
a_Si = 5.431  # Å

def d_si(h, k, l):
    """d-spacing for cubic: d = a_Si / sqrt(h^2 + k^2 + l^2)"""
    return a_Si / math.sqrt(h*h + k*k + l*l)

# GaN constants
a_GaN, c_GaN = 3.189, 5.185

def d_gan(h, k, i, l):
    """d-spacing for hexagonal: 1/d^2 = (4/3)*(h^2+hk+k^2)/a^2 + l^2/c^2"""
    part1 = (4/3)*(h*h + h*k + k*k) / (a_GaN*a_GaN)
    part2 = (l*l) / (c_GaN*c_GaN)
    return 1 / math.sqrt(part1 + part2)

# ---- Normal vectors ----
def to_cart_si(h, k, l):
    """Cartesian normal for cubic plane"""
    return np.array([h, k, l])

def to_cart_gan(h, k, i, l):
    """Cartesian normal for hexagonal plane"""
    x = h + k/2 + i/2
    y = (math.sqrt(3)/2)*(k - i)
    z = (l * c_GaN / a_GaN)
    return np.array([x, y, z])

# ---- Angle calculation ----
def angle_between(v1, v2):
    """θ = arccos((v1·v2)/(||v1||||v2||))"""
    cosv = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    cosv = np.clip(cosv, -1.0, 1.0)
    return math.degrees(math.acos(cosv))

# ---- Zone axis calculation ----
def zone_axis(p1, p2, hexagonal=False):
    """
    Compute zone axis:
    - cubic: cross product of 3-index vectors
    - hexagonal: algebraic cross of 4-index planes: (h,k,i,l)
    returns tuple of ints
    """
    if not hexagonal:
        v1 = to_cart_si(*p1)
        v2 = to_cart_si(*p2)
        z = np.cross(v1, v2)
        if np.allclose(z, 0): return (0,0,0)
        zi = np.round(z).astype(int)
        g = abs(zi[0])
        for comp in zi[1:]: g = math.gcd(g, abs(comp))
        return tuple((zi//g).tolist())
    else:
        h1,k1,i1,l1 = p1
        h2,k2,i2,l2 = p2
        U = k1*l2 - l1*k2
        V = l1*h2 - h1*l2
        W = h1*k2 - k1*h2
        T = -(U + V)
        g = abs(U)
        for comp in (V, T, W): g = math.gcd(g, abs(comp))
        if g == 0: g = 1
        return (U//g, V//g, T//g, W//g)

# ---- Build reference DataFrame ----
def build_ref(plane_list, d_func, cart_func, hexagonal=False):
    """Return DataFrame of reference pairs"""
    records = []
    planes = [(p, d_func(*p)) for p in plane_list]
    for (p1, d1), (p2, d2) in product(planes, repeat=2):
        if p1 == p2: continue
        ratio = d1 / d2
        v1 = cart_func(*p1)
        v2 = cart_func(*p2)
        ang = angle_between(v1, v2)
        za = zone_axis(p1, p2, hexagonal)
        records.append({'p1': p1, 'p2': p2, 'd1': d1, 'd2': d2,
                        'ratio': ratio, 'angle': ang, 'za': za})
    return pd.DataFrame(records)

# ---- Find best match ----
def find_best(df, r, a):
    df['er'] = abs(df['ratio'] - r)
    df['ea'] = abs(df['angle'] - a)
    df['tot'] = df['er'] + df['ea']
    return df.loc[df['tot'].idxmin()]

# ---- Main interactive app ----
def run_app():
    choice = input("Choose structure (Si/GaN): ").strip()
    hexagon = (choice == 'GaN')
    if hexagon:
        ref_df = build_ref(gan_planes, d_gan, to_cart_gan, hexagonal=True)
        fmt = format_hkil
    else:
        ref_df = build_ref(si_planes, d_si, to_cart_si, hexagonal=False)
        fmt = format_hkl
    print(f"\n{choice} SAED Indexing App. Type 'exit' to quit.")
    while True:
        ri = input("Measured ratio: ")
        if ri.lower() == 'exit': break
        ai = input("Measured angle: ")
        if ai.lower() == 'exit': break
        try:
            r_meas = float(ri); a_meas = float(ai)
        except ValueError:
            print("Invalid number, try again."); continue
        best = find_best(ref_df.copy(), r_meas, a_meas)
        p1,p2 = best['p1'], best['p2']
        print(f"\nMatch: {fmt(*p1)} ↔ {fmt(*p2)}")
        print(f"d1={best['d1']:.3f} Å, d2={best['d2']:.3f} Å, ratio={best['ratio']:.4f}")
        print(f"Err_ratio={best['er']:.4f}, Err_angle={best['ea']:.2f}°, Total_err={best['tot']:.4f}")
        print(f"Zone axis: {best['za']}\n")

if __name__=='__main__':
    # Precompute gan_planes after definition
    gan_planes = gan_basic
    si_planes = si_planes
    run_app()
